In [1]:
import requests
import csv
import time
from bs4 import BeautifulSoup

# Danh sách công ty và mã tương ứng
companies = {
    # Nông nghiệp
    "Hoàng Anh Gia Lai": "HAG",
    "Tập đoàn PAN": "PAN",
    "Dabaco Việt Nam": "DBC",
    "Giống cây trồng Trung Ương": "NSC",
    "Mía đường Lam Sơn": "LSS",
    "Mía đường Thành Thành Công Tây Ninh": "SBT",
    "Mía đường Sơn La": "SLS",
    "Mía đường Sóc Trăng": "SLS",
    "Thực phẩm Sao Ta": "FMC",
    "Nông nghiệp Quốc tế Hoàng Anh Gia Lai": "HNG",
    "Thực phẩm Cholimex": "CMF",
    "Giống cây trồng Miền Nam": "SSC",
    "Công ty cổ phần Vĩnh Hoàn": "VHC",
    "Công ty cổ phần Nam Việt": "ANV",
    "Công ty cổ phần Thủy sản Minh Phú": "MPC",
    "Công ty cổ phần Thực phẩm Bích Chi": "BFC",
    "Công ty cổ phần Tập đoàn Lộc Trời": "LTG",
    "CTCP Nông nghiệp BaF Việt Nam": "BAF",
    "CTCP Dược phẩm Cửu Long": "DCL",
    "CTCP Thuốc sát trùng Việt Nam": "VFG",
    "CTCP XNK Thủy sản Bến Tre": "ABT",
    "CTCP XNK An Giang": "AGM",
    "CTCP Chế biến Gỗ Đức Thành": "GDT",
    "CTCP Tập đoàn Dược Bảo Châu": "BCP",
    "CTCP Tập đoàn Sao Mai": "ASM",
    "CTCP Dầu thực vật Tường An": "TAC",
    "CTCP Dầu thực vật Cái Lân": "VOC"
}

years = list(range(2019, 2025))

# Hàm tải nội dung với retry
def get_with_retry(url, max_retries=5, wait=3):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
    }

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            print(f"    ✅ Tải thành công sau {attempt} lần thử")
            return response
        except requests.exceptions.HTTPError as e:
            print(f"    ⚠️ Lỗi HTTP {response.status_code} - thử lại sau {wait} giây (lần {attempt})")
        except requests.exceptions.RequestException as e:
            print(f"    ❌ Lỗi kết nối: {e} - thử lại sau {wait} giây (lần {attempt})")

        time.sleep(wait)

    print("    ❌ Không thể tải trang sau nhiều lần thử.")
    return None

# Bắt đầu xử lý từng công ty
for company_name, ticker in companies.items():
    print(f"🔍 Đang xử lý công ty: {company_name} ({ticker})")

    final_table = []

    for year_index, year in enumerate(years):
        url = f'https://cafef.vn/du-lieu/bao-cao-tai-chinh/{ticker}/bsheet/{year}/4/0/0/bao-cao-tai-chinh-.chn'
        print(f"  -> Năm {year}")

        response = get_with_retry(url)
        if response is None:
            continue  # bỏ qua năm này nếu không tải được

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', id='tableContent')

        if not table:
            print(f"    ❌ Không tìm thấy bảng dữ liệu")
            continue

        rows = table.find_all('tr')
        current_year_data = []

        for i, row in enumerate(rows):
            cols = row.find_all('td')
            if not cols:
                continue

            if len(cols) > 8:
                cols = cols[:-8]
            else:
                continue

            row_data = [col.text.strip() for col in cols]

            if len(row_data) < 2:
                continue

            if i == 0:
                if year_index == 0:
                    for q in range(1, len(row_data)):
                        if row_data[q] == '':
                            row_data[q] = f"Quý {q} năm {year}"
                    final_table.append(row_data)
                else:
                    new_titles = [f"Quý {q} năm {year}" for q in range(1, len(row_data))]
                    final_table[0].extend(new_titles)
            else:
                if year_index == 0:
                    current_year_data.append(row_data)
                else:
                    current_year_data.append(row_data[1:])  # bỏ cột đầu (chỉ tiêu)

        if year_index == 0:
            final_table.extend(current_year_data)
        else:
            for i in range(1, len(final_table)):
                final_table[i].extend(current_year_data[i - 1])

    # Ghi ra file CSV
    output_file = f"report 1/Tổng hợp báo cáo {company_name}.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(final_table)

    print(f"✅ Đã lưu vào file: {output_file}\n")


🔍 Đang xử lý công ty: Hoàng Anh Gia Lai (HAG)
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2022
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2023
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2024
    ✅ Tải thành công sau 1 lần thử
✅ Đã lưu vào file: report 1/Tổng hợp báo cáo Hoàng Anh Gia Lai.csv

🔍 Đang xử lý công ty: Tập đoàn PAN (PAN)
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2022
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2023
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2024
    ✅ Tải thành công sau 1 lần thử
✅ Đã lưu vào file: report 1/Tổng hợp báo cáo Tập đoàn PAN.csv

🔍 Đang xử lý công ty: Dabaco Việt Nam (DBC)
  -> Năm 2019
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2020
    ✅ Tải thành công sau 1 lần thử
  -> Năm 2021
    ✅ Tải thành công sau 1 lần thử
  -